In [1]:
#import library
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("dataset/MNIST/", one_hot=True)

Extracting dataset/MNIST/train-images-idx3-ubyte.gz


Extracting dataset/MNIST/train-labels-idx1-ubyte.gz
Extracting dataset/MNIST/t10k-images-idx3-ubyte.gz
Extracting dataset/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
X = tf.placeholder("float32", [None, 784], name="input")
Y = tf.placeholder("float32", [None, 10], name="target")

cell = tf.contrib.rnn.BasicLSTMCell(128)
cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.4)

fw = tf.get_variable("fw", shape=[128,10], dtype="float32", initializer=tf.truncated_normal_initializer(0.0,stddev=0.01), trainable=True)
fb = tf.get_variable("fb", shape=[10], dtype="float32", initializer=tf.constant_initializer(0.0), trainable=True)

cell = tf.contrib.rnn.MultiRNNCell([cell for _ in range(3)] )

ValueError: Variable fw already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Users\Kiyoung\AppData\Local\Programs\Python\Python35\lib\site-packages\tensorflow\python\framework\ops.py", line 1228, in __init__
    self._traceback = _extract_stack()
  File "C:\Users\Kiyoung\AppData\Local\Programs\Python\Python35\lib\site-packages\tensorflow\python\framework\ops.py", line 2336, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Users\Kiyoung\AppData\Local\Programs\Python\Python35\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 768, in apply_op
    op_def=op_def)


In [5]:
X_reshape = tf.reshape(X, [-1, 28, 28])

outputs, states = tf.nn.dynamic_rnn(cell, X_reshape, initial_state=None, dtype=tf.float32)

out = tf.matmul(outputs[:,-1], fw) + fb

loss_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=out))
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_func)

In [6]:
step = 0
train_iters = 100000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    correct_pred = tf.equal(tf.argmax(out, 1), tf.argmax(Y, 1))
    acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    while step < train_iters:
        batch_x, batch_y = mnist.train.next_batch(100)
        feed_dict = {X:batch_x, Y:batch_y}

        sess.run(optimizer, feed_dict=feed_dict)
        step += 1
        if step % 1000 == 0:
            racc = sess.run(acc, feed_dict=feed_dict)
            rloss = sess.run(loss_func, feed_dict=feed_dict)
            print(racc, rloss)

0.98 0.0969918


0.97 0.116863


1.0 0.0200942


0.98 0.104351


1.0 0.00939758


...

0.42 1.78272


0.33 1.7627


0.33 1.88963


0.31 1.72347


0.25 1.97799
